In [ ]:
#uses DotProduct_#N_S#noepsE_HRx#hrf_C#mamcs_A#baseAdderSize - v4.5

In [ ]:
import sys
import math
import os

In [ ]:
import import_ipynb
from DotProduct_SoE_V4_5 import DotProduct_SoE

In [ ]:
def MatrixVectorMultiply_SoE(parameters, Print_To_File, Files_Location, Is_Top, modules):

    M = parameters["M"]
    N = parameters["N"]
    PR = parameters["PR"]
    NMult = parameters["NOEPS"]
    MAMCS = parameters["MAMCS"]
    AdderSize = parameters["ADDER_SIZE"]
    Enable_Vector_Latch = parameters["ENABLE_COLUMN_LATCH"]

    #IN_WIDTH>=1
    IN_WIDTH = 10
    #INPUT_REG_DEPTH>=0
    INPUT_REG_DEPTH = 1
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = 1

    HRR = math.ceil(N/NMult);
    RS = math.ceil(M/PR)
    #NumOfComponents = math.ceil(NMult/MAMCS)

    ModuleName="MatrixVectorMultiply_{}_{}_{}PR_S{}E_HRx{}_C{}_A{}_".format(M, N, PR, NMult, HRR, MAMCS, AdderSize)
    if Enable_Vector_Latch<=0:
        ModuleName+="N"
    ModuleName+="VL"

    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)

    if Print_To_File<=0:
        of = sys.stdout
    else:
        of = open(Files_Location+ModuleName+".v", 'w+')

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
    print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
    print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("output newInSeriesStart,", file=of)
    lghrr=math.ceil(math.log2(HRR))
    if lghrr==1:
        print("output inSeries,", file=of)
    elif lghrr>1:
        print("output [{}:0] inSeries,".format(lghrr-1), file=of)
    lgRS=math.ceil(math.log2(RS))
    if lgRS<=1:
        print("output reg rowSetInNo = 0,", file=of)
    else:
        print("output reg [{}:0] rowSetInNo = 0,".format(lgRS-1), file=of)
    print("input inReady,", file=of)
    for j in range(PR):
        print("input signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(NMult):
            print("AE{}R{}, ".format(i, j), end='', file=of)
        print(file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NMult):
        print("VE{}, ".format(i), end='', file=of)
    print(file=of)
    ar=RS*PR-M
    ME = PR-ar-1
    if ME==0:
        st1="E0"
    else:
        st1="E0toE{}".format(ME)
    print("output ", end='', file=of)
    print(st1, end='', file=of)
    print("Ready,", file=of)
    if ar==0:
        print("output reg NUCReady = 0, //not used", file=of)
    else:
        if ME==(PR-2):
            st2="E{}".format(PR-1)
        else:
            st2="E{}toE{}".format(ME+1, PR-1)
        print("output ", end='', file=of)
        print(st2, end='', file=of)
        print("Ready,", file=of)
    if lgRS<=1:
        print("output reg rowSetOutNo = {},".format(RS-1), file=of)
    else:
        print("output reg [{}:0] rowSetOutNo = {},".format(lgRS-1, RS-1), file=of)
    lgn=math.ceil(math.log2(N))
    al=-1+lgn
    if al<0:
        print("output signed [(2*IN_WIDTH){}:0] ".format(al), end='',file=of)
    elif al==0:
        print("output signed [(2*IN_WIDTH):0] ", end='',file=of)
    else:
        print("output signed [(2*IN_WIDTH)+{}:0] ".format(al), end='',file=of)
    for i in range(PR):
        print("E{}, ".format(i), end='',file=of)
    print(file=of)
    print("output early", end='', file=of)
    print(st1, end='', file=of)
    print("Ready,", file=of)
    if ar==0:
        print("output reg NUearlyCReady = 0 //not used", file=of)
    else:
        print("output early", end='', file=of)
        print(st2, end='', file=of)
        print("Ready", file=of)
    print(");\n", file=of)

    if Enable_Vector_Latch>0:
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(NMult-1):
            print("SVE{}, ".format(i), end='', file=of)
        print("SVE{};".format(NMult-1), file=of)
        print(file=of)

    if(ar>0): #RS*PR > M, RS>=2
        print("wire inReadyC = (rowSetInNo!={}) & inReady;".format(RS-1), file=of)
        print(file=of)
    for i in range (PR):
        print("DotProduct_{}_S{}E_HRx{}_C{}_A{}".format(N, NMult, HRR, MAMCS, AdderSize), file=of)
        print("#( .IN_WIDTH(IN_WIDTH), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
        print("DP_{}(clk, reset, enable,".format(i), file=of)
        if i==0:
            print("newInSeriesStart,", file=of)
            print("inSeries,".format(i), file=of)
        else:
            print("newInSeriesStart{}, //not used".format(i), file=of)
            print("inSeries{}, //not used".format(i), file=of)
        if(PR-i<=ar):
            print("inReadyC,", file=of)
        else:
            print("inReady,", file=of)
        for j in range(NMult):
            print("AE{}R{}, ".format(j, i), end='', file=of)
        print(file=of)
        if Enable_Vector_Latch>0:
            for j in range(NMult):
                print("SVE{}, ".format(j), end='', file=of)
        else:
            for j in range(NMult):
                print("VE{}, ".format(j), end='', file=of)        
        print(file=of)
        if i==0:
            print(st1+"Ready,", file=of)
        elif i==PR-ar:
            print(st2+"Ready,", file=of)
        else:
            print("outReady{}, //not used".format(i), file=of)
        print("E{},".format(i), file=of)
        if i==0:
            print("early"+st1+"Ready);", file=of)
        elif i==PR-ar:
            print("early"+st2+"Ready);", file=of)
        else:
            print("earlyOutReady{}); //not used".format(i), file=of)
        print(file=of)

    if(RS>1):
        if Enable_Vector_Latch>0:
            print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
            for i in range(N-1):
                print("LVE{}, ".format(i), end='', file=of)
            print("LVE{};".format(N-1), file=of)
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\trowSetInNo <= 0;", file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tif(inReady) begin", file=of)
        print("\t\t\tif(inSeries=={}) begin".format(HRR-1), file=of)
        print("\t\t\t\tif(rowSetInNo=={}) begin".format(RS-1), file=of)
        print("\t\t\t\t\trowSetInNo <= 0;", file=of)
        print("\t\t\t\tend", file=of)
        print("\t\t\t\telse begin", file=of)
        if RS==2:
            print("\t\t\t\t\trowSetInNo <= 1;", file=of)    
        else:
            print("\t\t\t\t\trowSetInNo <= rowSetInNo+1;", file=of)
        print("\t\t\t\tend", file=of)
        print("\t\t\tend", file=of)
        if Enable_Vector_Latch>0:
            print("\t\t\tif(rowSetInNo==0) begin", file=of)
            print("\t\t\t\tcase(inSeries)", file=of)
            for i in range(HRR):
                print("\t\t\t\t\t{}: begin".format(i), file=of)
                print("\t\t\t\t\t\t", end='', file=of)
                k=NMult*i-1
                for j in range(NMult):
                    k=k+1
                    print("LVE{} <= VE{}; ".format(k,j), end='', file=of)
                    if k == N-1:
                        break
                print(file=of)
                print("\t\t\t\t\tend", file=of)
            print("\t\t\t\tendcase", file=of)
            print("\t\t\tend", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    if Enable_Vector_Latch>0:
        print("always @(*) begin", file=of)
        print("\tif(rowSetInNo==0) begin", file=of)
        print("\t\t", end='', file=of)
        for j in range(NMult):
            print("SVE{} = VE{}; ".format(j,j), end='', file=of)
        print(file=of)
        print("\tend", file=of)
        print("\telse begin", file=of)
        print("\t\tcase(inSeries)", file=of)
        for i in range(HRR):
            print("\t\t\t{}: begin".format(i), file=of)
            print("\t\t\t\t", end='', file=of)
            k=NMult*i-1
            for j in range(NMult):
                k=k+1
                if k >= N:
                    print("SVE{} = 0; ".format(j), end='', file=of)            
                else:
                    print("SVE{} = LVE{}; ".format(j,k), end='', file=of)
            print(file=of)
            print("\t\t\tend", file=of)
        if (HRR & (HRR-1)) != 0: #check if HRR is not power of 2
            print("\t\t\tdefault: begin".format(k), file=of)
            print("\t\t\t\t", end='', file=of)
            for j in range(NMult):
                print("SVE{} = 0; ".format(j), end='', file=of)
            print(file=of)
            print("\t\t\tend", file=of)
        print("\t\tendcase", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    if(RS>1):
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\trowSetOutNo <= {};".format(RS-1), file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tif(early"+st1+"Ready) begin", file=of)
        print("\t\t\tif(rowSetOutNo=={}) begin".format(RS-1), file=of)
        print("\t\t\t\trowSetOutNo <= 0;", file=of)
        print("\t\t\tend", file=of)
        print("\t\t\telse begin", file=of)
        if RS==2:
            print("\t\t\t\trowSetOutNo <= 1;", file=of)
        else:
            print("\t\t\t\trowSetOutNo <= rowSetOutNo+1;", file=of)
        print("\t\t\tend", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()

    DotProduct_SoE(parameters, Print_To_File, Files_Location, 0, modules)

    modules[ModuleName]="MatrixVectorMultiply_SoE"
    return ModuleName